## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [1]:
from keras.models import Sequential
from keras.layers import Conv2D 
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization, Activation
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder

Using TensorFlow backend.


In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7) 
        return X_train, X_test,mean,std
    
    
## Normalize Training and Testset    
x_train, x_test,mean_train,std_train = normalize(x_train, x_test) 

(50000, 32, 32, 3)


In [3]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

C:\Users\doggy\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [4]:
classifier=Sequential()

#卷積組合
classifier.add(Conv2D(32, kernel_size=(3,3), input_shape=(32,32,3), padding='same', activation='relu'))
classifier.add(BatchNormalization(momentum=0.99, epsilon=0.001))

classifier.add(Conv2D(64, kernel_size=(3,3), padding='same', activation='relu'))
classifier.add(BatchNormalization(momentum=0.99, epsilon=0.001))

classifier.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

#flatten
classifier.add(Flatten())

#FC
classifier.add(Dense(128))
classifier.add(Activation('relu'))
#classifier.add(Dropout(0.5))

#輸出
classifier.add(Dense(output_dim=10,activation='softmax'))

#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.fit(x_train,y_train,batch_size=100,epochs=20)

C:\Users\doggy\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=10)`


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/20
50000/50000 [==============================] - 408s 8ms/step - loss: 1.4826 - acc: 0.4971
Epoch 2/20
50000/50000 [==============================] - 441s 9ms/step - loss: 0.9865 - acc: 0.6517
Epoch 3/20
50000/50000 [==============================] - 431s 9ms/step - loss: 0.8058 - acc: 0.7183
Epoch 4/20
50000/50000 [==============================] - 402s 8ms/step - loss: 0.6752 - acc: 0.7670
Epoch 5/20
50000/50000 [==============================] - 407s 8ms/step - loss: 0.5452 - acc: 0.8109
Epoch 6/20
50000/50000 [==============================] - 406s 8ms/step - loss: 0.4186 - acc: 0.8540
Epoch 7/20
50000/50000 [==============================] - 408s 8ms/step - loss: 0.3055 - acc: 0.8946
Epoch 8/20
50000/50000 [==============================] - 348s 7ms/step - loss: 0.2283 - acc: 0.9204
Epoch 9/20
50000/50000 [==============================] - 347s 7ms/step - loss: 0.1734 - acc: 0.94

In [5]:
scores = classifier.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

10000/10000 [==============================] - 28s 3ms/step
Test loss: 2.3248674547195436
Test accuracy: 0.6761


## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [15]:
import cv2

img = cv2.imread('dog_test.jpg', cv2.IMREAD_COLOR) # 以彩色圖片的方式載入
print(img.shape)

pic = cv2.resize(img, (32, 32), interpolation=cv2.INTER_CUBIC)
pic = pic.reshape(-1,32,32,3)
print(pic.shape)
#cv2.imshow('', pic)
#cv2.waitKey(0)
#cv2.destroyAllWindows()

(319, 512, 3)
(1, 32, 32, 3)


In [16]:
input_example=(pic-mean_train)/(std_train+1e-7) 
classifier.predict(input_example)

array([[4.5847426e-07, 1.6540615e-04, 5.3530028e-12, 2.2788903e-11,
        5.3815952e-16, 6.8753232e-11, 1.5503820e-15, 2.8481281e-10,
        1.2346993e-10, 9.9983406e-01]], dtype=float32)

In [17]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
classifier.predict(input_example)

array([[2.1252190e-06, 1.5971089e-17, 9.4313123e-07, 9.9990773e-01,
        3.3809196e-05, 2.3069142e-21, 1.2436841e-07, 1.6817286e-19,
        5.5162614e-05, 2.8269307e-11]], dtype=float32)